# [Assignment #2: NPFL067 Statistical NLP II](http://ufal.mff.cuni.cz/~hajic/courses/npfl067/assign2.html)

## Words and The Company They Keep

### Author: Dan Kondratyuk

### March 2, 2018

---

This Python notebook examines 

Code and explanation of results is fully viewable within this webpage.

## Files

- [index.html](./index.html) - Contains all veiwable code and a summary of results
- [README.md](./README.md) - Instructions on how to run the code with Python
- [nlp-assignment-2.ipynb](./nlp-assignment-2.ipynb) - Jupyter notebook where code can be run
- [requirements.txt](./requirements.txt) - Required python packages for running

## 1. Best Friends

#### Problem Statement
>  In this task you will do a simple exercise to find out the best word association pairs using the pointwise mutual information method.

> First, you will have to prepare the data: take the same texts as in the previous assignment, i.e.

> `TEXTEN1.txt` and `TEXTCZ1.txt`

> (For this part of Assignment 2, there is no need to split the data in any way.)

> Compute the pointwise mutual information for all the possible word pairs appearing consecutively in the data, **disregarding pairs in which one or both words appear less than 10 times in the corpus**, and sort the results from the best to the worst (did you get any negative values? Why?) Tabulate the results, and show the best 20 pairs for both data sets.

> Do the same now but for distant words, i.e. words which are at least 1 word apart, but not farther than 50 words (both directions). Again, tabulate the results, and show the best 20 pairs for both data sets. 

### Process Text

In [1]:
# Import Python packages
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
# %load_ext autoreload
# %autoreload 2

from collections import defaultdict, Counter, Iterable
import itertools
import nltk
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm, tnrange as trange
from scipy.special import comb

# Configure Plots
plt.rcParams['lines.linewidth'] = 4
pd.set_option('max_colwidth', 100)

np.random.seed(200) # Set a seed so that this notebook has the same output each time

In [2]:
def open_text(filename):
    """Reads a text line by line, applies light preprocessing, and returns an array of words"""
    with open(filename, encoding='iso-8859-2') as f:
        content = f.readlines()
    
    preprocess = lambda word: word.strip()
    
    return np.array([preprocess(word) for word in content])

In [3]:
class LanguageModel:
    """Counts words and calculates the probabilities of a language model"""
    
    def __init__(self, words, min_words=10):
        self.min_words = min_words
        
        # Unigrams
        self.unigrams = words
        self.unigram_set = list(set(self.unigrams))
        self.total_unigram_count = len(self.unigrams)
        self.unigram_dist = Counter(self.unigrams)
        
        self.unigram_pdist = defaultdict(float)
        for w in self.unigram_dist:
            self.unigram_pdist[w] = self.unigram_dist[w] / self.total_unigram_count
        
        # Bigrams
        self.bigrams = list(nltk.bigrams(words))
        self.bigram_set = list(set(self.bigrams))
        self.total_bigram_count = len(self.bigrams)
        self.bigram_dist = Counter(self.bigrams)
        
        self.bigram_pdist = defaultdict(float)
        for w in self.bigram_dist:
            self.bigram_pdist[w] = self.bigram_dist[w] / self.total_bigram_count
    
    def p_unigram(self, w):
        """Calculates the probability a unigram appears in the distribution"""
        return self.unigram_pdist[w]
    
    def p_bigram(self, wprev, w):
        """Calculates the probability a bigram appears in the distribution"""
        return self.bigram_pdist[(wprev, w)]
    
    def pointwise_mi(self, wprev, w, p_bigram_func=None):
        """Calculates the pointwise mutual information in a word pair"""
        p_bigram_func = self.p_bigram if p_bigram_func is None else p_bigram_func
        joint = p_bigram_func(wprev, w)
        independent = self.p_unigram(wprev) * self.p_unigram(w)
        return np.log2(joint / independent) if independent != 0 else 0

In [4]:
# Read the texts into memory
english = './TEXTEN1.txt'
czech = './TEXTCZ1.txt'

words_en = open_text(english)
words_cz = open_text(czech)

In [5]:
lm_en = LanguageModel(words_en)
lm_cz = LanguageModel(words_cz)

In [6]:
def mutual_information(lm):
    # Obtain all word pairs in the word list, disregarding pairs in which one or both words appear less than 10 times in the corpus  
    pairs = [pair for pair in lm.bigram_set
             if lm.unigram_dist[pair[0]] >= lm.min_words 
             and lm.unigram_dist[pair[1]] >= lm.min_words]

    mi = [(' '.join(pair), lm.pointwise_mi(*pair)) for pair in pairs]
    return pd.DataFrame(mi, columns=['pair', 'mutual_information'])

In [7]:
mi_en = mutual_information(lm_en).sort_values(by='mutual_information', ascending=False)
mi_cz = mutual_information(lm_cz).sort_values(by='mutual_information', ascending=False)

In [8]:
mi_en[:20]

,pair,mutual_information
1812,La Plata,14.169370
10985,Asa Gray,14.031867
13301,Fritz Muller,13.362016
25145,worth while,13.332869
39601,faced tumbler,13.262480
13295,lowly organised,13.216899
28544,Malay Archipelago,13.110477
18174,shoulder stripe,13.053893
33993,Great Britain,12.914557
20439,United States,12.847442


In [9]:
mi_cz[:20]

,pair,mutual_information
14167,Hamburger SV,14.288950
33829,Los Angeles,14.062442
29292,Johna Newcomba,13.762882
15824,Č. Budějovice,13.633599
6077,série ATP,13.468968
9789,turnajové série,13.434411
37677,Tomáš Ježek,13.428981
22711,Lidové noviny,13.329922
4938,Lidových novin,13.271028
11320,veřejného mínění,13.062442


In [10]:
mi_en[:-5:-1]

,pair,mutual_information
5482,"the ,",-8.790285
5032,. the,-8.407455
7911,of .,-7.901950
9651,. of,-7.901950


In [11]:
def mutual_information_dist(lm):
    def mi_step(distance):
        # Get all pairs in the word list a certain distance apart
        pair_list = list(zip(lm.unigrams, lm.unigrams[distance+1:]))
        dist = Counter(pair_list)
    
        # Obtain all word pairs in the word list, disregarding pairs in which one or both words appear less than 10 times in the corpus  
        pairs = [pair for pair in list(set(pair_list))
                 if lm.unigram_dist[pair[0]] >= lm.min_words 
                 and lm.unigram_dist[pair[1]] >= lm.min_words]
        
        p_bigram = lambda wprev, w: dist[(wprev, w)] / lm.total_bigram_count
        
        yield ((distance, wprev, w, lm.pointwise_mi(wprev, w, p_bigram)) for wprev,w in pairs)
    
    max_distance = 50
    results = [m for distance in tqdm(range(1, max_distance+1)) for mi in mi_step(distance) for m in mi]
        
    return pd.DataFrame(results, columns=['distance', 'word0', 'word1', 'mutual_information'])

In [12]:
mi_dist_en = mutual_information_dist(lm_en).sort_values(by='mutual_information', ascending=False)
mi_dist_cz = mutual_information_dist(lm_cz).sort_values(by='mutual_information', ascending=False)

HBox(children=(IntProgress(value=0, max=50), HTML(value='')))

HBox(children=(IntProgress(value=0, max=50), HTML(value='')))

In [13]:
mi_dist_en[:20]

,distance,word0,word1,mutual_information
100747,2,survival,fittest,13.754333
34024,1,dimorphic,trimorphic,13.353454
109425,2,Alph,Candolle,13.236485
171136,3,H,Watson,13.169370
84829,2,Old,Worlds,13.053893
13541,1,Alph,de,13.053893
25956,1,E,Forbes,12.946978
134535,2,unimportant,welfare,12.879864
220813,3,carrier,faced,12.695439
56208,1,rarer,rarer,12.525514


In [14]:
mi_dist_cz[:20]

,distance,word0,word1,mutual_information
32730,1,ODÚ,VPN,14.119025
34986,1,turnajové,ATP,13.614983
26719,1,Mistrovství,turnajové,13.410365
388108,8,výher,výher,13.318097
47522,1,Čechy,Slováky,13.303450
139690,3,Mistrovství,ATP,13.203914
1000244,19,prohraná,dvojchyby,13.172205
87893,2,soužití,Slováků,13.062442
654498,13,prohraná,esa,13.051911
410973,8,III,IV,13.025916


## 2. Best Friends

#### Word Classes

> **The Data**

> Get `TEXTEN1.ptg`, `TEXTCZ1.ptg`. These are your data. They are almost the same as the .txt data you have used so far, except they now contain the part of speech tags in the following form:

> `rady/NNFS2-----A----`  
`,/Z:-------------`

> where the tag is separated from the word by a slash ('/'). Be careful: the tags might contain everything (including slashes, dollar signs and other weird characters). It is guaranteed however that there is no slash-word.

> Similarly for the English texts (except the tags are shorter of course).

> **The Task**

> Compute a full class hierarchy of **words** using the first 8,000 words of those data, and only for words occurring 10 times or more (use the same setting for both languages). Ignore the other words for building the classes, but keep them in the data for the bigram counts. For details on the algorithm, use the Brown et al. paper distributed in the class; some formulas are wrong, however, so please see the corrections on the web (Class 12, formulas for Trick \#4). Note the history of the merges, and attach it to your homework. Now run the same algorithm again, but stop when reaching 15 classes. Print out all the members of your 15 classes and attach them too.

> **Hints:**

> The initial mutual information is (English, words, limit 8000):

> `4.99726326162518` (if you add one extra word at the beginning of the data)  
> `4.99633675507535` (if you use the data as they are and are carefull at the beginning and end).

> NB: the above numbers are finally confirmed from an independent source :-).

> The first 5 merges you get on the English data should be:

> `case subject`  
> `cannot may`  
> `individuals structure`  
> `It there`  
> `even less`  

> The loss of Mutual Information when merging the words "case" and "subject":

> Minimal loss: `0.00219656653357569` for `case+subject`

In [2]:
def open_text(filename):
    """Reads a text line by line, applies light preprocessing, and returns an array of words and tags"""
    with open(filename, encoding='iso-8859-2') as f:
        content = f.readlines()
    
    preprocess = lambda word: word.strip().rsplit('/', 1)
    
    return [preprocess(word) for word in content]

In [3]:
# Read the texts into memory
english = './TEXTEN1.ptg'
czech = './TEXTCZ1.ptg'

words_en, tags_en = zip(*open_text(english))
words_cz, tags_cz = zip(*open_text(czech))

In [4]:
class LmCluster:
    def __init__(self, words, word_cutoff=10):
        self.word_cutoff = word_cutoff
        
        # Unigrams
        self.text_size = len(words)
        self.word2int = {}
        self.unigram_dist = defaultdict(int)
        
        word_counts = Counter(words)
        word_set = sorted(word_counts, key=lambda w: word_counts[w], reverse=True)
        
        for i, w in enumerate(word_set):
            self.word2int[w] = i
            self.unigram_dist[i] = word_counts[w]
        
        self.int2word = sorted(self.word2int, key=lambda word: self.word2int[word])
        self.unigrams = [self.word2int[w] for w in words]
        
        # Bigrams
        self.bigrams = list(zip(self.unigrams, self.unigrams[1:]))
        self.bigram_dist = defaultdict(lambda: defaultdict(int))
        for wprev, w in self.bigrams:
            self.bigram_dist[wprev][w] += 1
        
        self.classes = [word for word in self.unigram_dist if self.unigram_dist[word] >= self.word_cutoff]
        self.class_counter = len(self.unigram_dist)
        
        self.merge_history = []
        
    def cluster(self, class_count=1):
        merges = len(self.classes) - class_count
        for _ in trange(merges, unit='class'):
            mi, l, r = self.best_merge()
            c_new = self.merge(l, r)
            
            save = (*self.class_name([l, r]), c_new, mi)
            self.merge_history.append(save)
            
            print(save)

    def best_merge(self):
        s = defaultdict(float)
        for c in self.classes:
            s[c] += np.sum(self.mutual_information([a], [c]) for a in self.bigram_dist)
            s[c] += np.sum(self.mutual_information([c], [b]) for b in self.bigram_dist)
            s[c] -= self.mutual_information([c], [c])

        mi = (self.merge_mi(l, r, s) for l, r in itertools.combinations(self.classes, 2))
        progress = tqdm(mi, total=comb(len(self.classes), 2, exact=True), leave=False)
        return max(progress, key=lambda x: x[0])

    def merge_mi(self, l, r, s):
        mi = 0.0
        mi -= s[l] + s[r]
        mi += self.mutual_information([l], [r])
        mi += self.mutual_information([r], [l])
        mi += self.mutual_information([l, r], [l, r])

        for c in self.bigram_dist:
            if c in [l, r]:
                continue
            mi += self.mutual_information([l, r], [c])
            mi += self.mutual_information([c], [l, r])

        return mi, l, r
    
    def merge(self, l, r):
        c_new = self.class_counter
        
        # Add the new class to frequency distributions
        self.unigram_dist[c_new] = self.unigram_dist[l] + self.unigram_dist[r]
        
        for c in [l, r]:
            for d, count in self.bigram_dist[c].items():
                d = c_new if d in [l, r] else d
                self.bigram_dist[c_new][d] += count
        
        for c in self.bigram_dist:
            for d in [l, r]:
                if d in self.bigram_dist[c] and c != c_new:
                    self.bigram_dist[c][c_new] += self.bigram_dist[c][d]

        # Update classes
        for c in [l, r]:
            self.classes.remove(c)
        self.classes.append(c_new)
        
        self.class_counter += 1
        
        return c_new
    
    def mutual_information_total(self):
        return np.sum(self.mutual_information([wprev], [w]) for wprev in self.bigram_dist for w in self.bigram_dist[wprev])

    def mutual_information(self, left, right):
        bigram_count = np.sum(self.bigram_dist[l][r] for l in left for r in right)
        
        if not bigram_count:
            return 0.0
        
        left_count  = np.sum(self.unigram_dist[c] for c in left)
        right_count = np.sum(self.unigram_dist[c] for c in right)
        
        return (bigram_count / self.text_size) * np.log2(bigram_count * self.text_size / left_count / right_count)
    
    def class_name(self, classes):
        if not isinstance(classes, Iterable):
            classes = [classes]

        classes = [self.int2word[c] if c < len(self.int2word) else c for c in classes]
        return classes if len(classes) > 1 else classes[0]

In [6]:
text_size = 8000

In [ ]:
lm_en = LmCluster(words_en[:text_size])
lm_en.cluster()

In [ ]:
lm_en = LmCluster(words_en[:text_size])
lm_cz = LmCluster(words_cz[:text_size])

In [ ]:
lm_en.cluster()
lm_cz.cluster()

In [24]:
def history(cluster):
    return pd.DataFrame(cluster.merge_history, columns=['class 1', 'class 2', 'cluster id'])

In [ ]:
history(lm_en)[:20]

In [ ]:
history(lm_cz)[:20]

In [ ]:
lm_en_15 = LmCluster(words_en[:8000])
lm_cz_15 = LmCluster(words_cz[:8000])

In [25]:
def get_classes(cluster):
    classes = c.defaultdict(list)

    for c0 in cluster.classes:
        for w in cluster.vocab:
            cur_cluster = cluster.vocab[w]
            while cur_cluster in cluster.cluster_parents:
                cur_cluster = cluster.cluster_parents[cur_cluster]
            if c0 == cur_cluster:
                classes[c0].append(w)

    return pd.DataFrame([(x,classes[x]) for x in classes], columns=['class', 'words'])

In [26]:
get_classes(lm_en_15)

,class,words
0,0,"[,]"
1,1,[the]
2,2,[of]
3,3,[and]
4,4,[.]
5,8,[a]
6,10,[I]
7,12,[as]
8,13,[be]
9,14,[have]


In [27]:
get_classes(lm_cz_15)

,class,words
0,0,[.]
1,1,"[,]"
2,2,[a]
3,3,[v]
4,4,[se]
5,6,[o]
6,7,"[""]"
7,9,[že]
8,14,[je]
9,15,[i]


## 3. Tag Classes

> Use the same original data as above, but this time, you will compute the classes for tags (the strings after slashes). Compute tag classes for all tags appearing 5 times or more in the data. Use as much data as time allows. You will be graded relative to the other student's results. Again, note the full history of merges, and attach it to your homework. Pick three interesting classes as the algorithm goes (English data only; Czech optional), and comment on them (why you think you see those tags there together (or not), etc.). 

In [40]:
cluster_en_tag = LmCluster(tags_en, word_cutoff=5)
cluster_cz_tag = LmCluster(tags_cz[:len(tags_cz)], word_cutoff=5)

2018-03-03 17:12:23,725	221098 word tokens were processed.
2018-03-03 17:12:23,725	Starting classes: 36
2018-03-03 17:12:23,725	initializing tables
100%|██████████| 35/35 [00:00<00:00, 309.61class/s]
2018-03-03 17:12:24,016	224538 word tokens were processed.
2018-03-03 17:12:24,016	Starting classes: 677
2018-03-03 17:12:24,017	initializing tables
100%|██████████| 676/676 [10:31<00:00,  1.07class/s]


In [43]:
def history(cluster):
    return pd.DataFrame(cluster.cluster_history, columns=['prev tag', 'tag', 'cluster id'])

In [44]:
history(cluster_en_tag)[:20]

,prev tag,tag,cluster id
0,RBR,WP$,36
1,JJS,36,37
2,SYM,NNPS,38
3,PRP,EX,39
4,NN,38,40
5,37,40,41
6,FW,41,42
7,RB,"""",43
8,(,42,44
9,NNS,WP,45


In [45]:
history(cluster_cz_tag)[:20]

,prev tag,tag,cluster id
0,AAIP6----3A----,CrIP6----------,1015
1,J^------------8,NNNXX-----A---8,1016
2,PSFS6-P1-------,AAFS6----2A----,1017
3,AGFS7-----A----,PSFS7-P1-------,1018
4,Vi-P---1--N----,PJXP2----------,1019
5,AAIP3----3A----,AGIP3-----A----,1020
6,PZXP6----------,PSXP6-P1-------,1021
7,PLXP6----------,AGFP6-----A----,1022
8,AAFP7----2A----,AAFP7----1N----,1023
9,AAIS6----1N----,AAIS6----3A----,1024
